In [4]:
import argparse
import os

from filelock import FileLock
from tensorflow.keras.datasets import mnist

import ray
from ray import train, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.air.integrations.keras import ReportCheckpointCall

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
!sudo pip install tensorflow==2.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 MB 2.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 99.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 102.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.4/462.4 kB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 87.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 96.4 MB/s eta 0:00:00
  Using cached tensorboard-2.14.0-py3-none-any.whl (5.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 107.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━

In [3]:
import tensorflow

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
import torchmetrics as tm

In [ ]:
import os
import tempfile
import time

import mlflow
from ray import air, tune
from ray.air import session
from domino_mlflow_utils.mlflow_callback import *

In [ ]:
def evaluation_fn(step, width, height):
    return (0.1 + width * step / 100) ** (-1) + height * 0.1


def train_function(config):
    width, height = config["width"], config["height"]

    for step in range(config.get("steps", 100)):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back to Tune.
        session.report({"iterations": step, "mean_loss": intermediate_score})
        time.sleep(0.1)

In [ ]:
def tune_with_callback(mlflow_tracking_uri, experiment_name,finish_fast=False):
    cb = MyMLflowLoggerCallback(
                    tracking_uri=mlflow_tracking_uri,
                    experiment_name=experiment_name,
                    save_artifact=True,
                )
    tuner = tune.Tuner(
        train_function,
        tune_config=tune.TuneConfig(num_samples=5),        
        run_config=air.RunConfig(
            name="mlflow",
            callbacks=[
                cb
            ],
        ),
        param_space={
            "width": tune.randint(10, 100),
            "height": tune.randint(0, 100),
            "steps": 5 if finish_fast else 100,
        },
    )
    results = tuner.fit()
    cb.log_end_parent_run()

In [ ]:
import mlflow
import os
import ray
import logging
from domino_mlflow_utils.mlflow_callback import *

temp_dir='/tmp'

if not ray.is_initialized():
    service_host = os.environ["RAY_HEAD_SERVICE_HOST"]
    service_port = os.environ["RAY_HEAD_SERVICE_PORT"]
    address=f"ray://{service_host}:{service_port}"
    #temp_dir='/mnt/data//{}/'.format(os.environ['DOMINO_PROJECT_NAME']) #set to a dataset
    ray.init(address=address, _temp_dir=temp_dir,runtime_env={"py_modules": ['/mnt/code/domino_mlflow_utils']})

print('Ray Initializied')
print(f'Ray Host={service_host} and Ray Port={service_port}')

experiment_name = 'RAYv7'+'-' + os.environ['DOMINO_STARTING_USERNAME'] + '-' + os.environ['DOMINO_PROJECT_NAME']
tune_with_callback(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)

#tune_with_setup(os.environ['CLUSTER_MLFLOW_TRACKING_URI'], experiment_name,finish_fast=True)
df = mlflow.search_runs(
        [mlflow.get_experiment_by_name(experiment_name).experiment_id]
    )
print(df)
